It important to keep backing up your data in external memory devices at the regular interval. Since i don't have a lot of multimedia data, i use a 256 GB mirco sd card for my backups .But usually we don't have enough to time to go through all of our files and copy only the files that are modified since the last backup. Plus, there are a few file/folder that i don't need to backup, but i still end up copying them anyways. This is both, time consuming and space consuming. 

A solution is to automate the backup process to make it more streamlined and less space and time consuming. One way to achieve this , which i am going to implement here is as follows. Keep track of the last time you took backup. specify all the folder that needs to be copied. go over through the folder and identify files that were modified after the last backup date. copy all the updated files in the new folder name in the external memory device. upadate the last backup time stamp and we are done 

**Prerequisites**

- understand how external devices are mounted in the the file system ( i am using macOS)
- understand how to access interal and external files in Python
- how to access file metadata in pyhon
- create a config file, read/update the file after every backup

**todo later**
- analytics : how many new files updates, compare with older backups .... 